In [1]:
import torch
from NetMF import NetMF
from Netmfutils import Netmfutils
import scipy.io
import numpy as np
import networkx as nx
import cogdl.datasets as cd
import scipy.sparse as sp

Using backend: pytorch


In [2]:
if torch.cuda.is_available():
        torch.cuda.set_device(0)

import logging
from cogdl.datasets import build_dataset
logger = logging.getLogger(__name__)
netutils = Netmfutils()

In [9]:
Label_repo = "dataset/blogcatalog.mat"
Rank = 128
Dim = 128
Window = 1
Large_netmf = False
Rand_Seed = 1
Neg_sampling = 1

Start_train_ratio = 10
Stop_train_ratio = 90
Num_train_ratio = 9
C = 1
Num_split = 5
Train_ratio = 80

In [10]:
"""
# load dataset from .mat
data = scipy.io.loadmat(Label_repo)
logger.info("Loading label from %s...", Label_repo)
# load label from .mat
label = netutils.load_label(file=Label_repo, variable_name="group")
logger.info("Label loaded!")
"""
dataset = cd.build_dataset_from_name('blogcatalog')
data = dataset[0]
G = nx.Graph()
G.add_edges_from(data.edge_index.t().tolist())
label_matrix = sp.csr_matrix(data.y).toarray().astype(np.int)
print (label_matrix.shape)

(10312, 39)


In [11]:
netmf = NetMF(Dim,Window,Rank,Neg_sampling,Large_netmf)
embedding = netmf.train(G,Graph_input = True)
netutils.node_classification_loss(embedding, label_matrix, train_ratio=Train_ratio/100.,
                                  n_splits=Num_split, C=C, random_state=Rand_Seed)


Running NetMF for a small window size...
Compute matrix %d-th power 1
(8249,) (2063,)
(10312, 128) (10312, 39)
(8249,) (2063,)
(10312, 128) (10312, 39)
(8249,) (2063,)
(10312, 128) (10312, 39)
(8249,) (2063,)
(10312, 128) (10312, 39)
(8249,) (2063,)
(10312, 128) (10312, 39)
%d fold validation, training ratio %f 5 0.8
Average micro %.2f, Average macro %.2f 16.11835444614371 3.7641255944232137


/usr/local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1492: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


([0.16562282533054976,
  0.1752400548696845,
  0.15809327846364885,
  0.15690946221919674,
  0.1500521014241056],
 [0.036844834191508975,
  0.04554122560311246,
  0.03861440924442194,
  0.03593888096285791,
  0.031266929719259366])